## 단계

이 노트북은 Amazon Personalize의 새로운 사용자 개인화 레시피(aws-user-personalization)를 사용하는 방법을 보여줍니다. 이 레시피는 새로운 항목과 이전 항목 간에 추천을 균형 있게 조정하여, 새 항목이나 이전 항목을 더 많이 추천하도록 조정할 수 있게 합니다.

새로운 USER Personalization 레시피를 사용하는 개략적인 단계는 다음과 같습니다.

1. Personalize 클라이언트를 설정합니다.
2. 데이터 세트 그룹을 만들고, 스키마를 정의하고, 데이터 세트를 가져오고, 실시간 상호 작용을 수집합니다.
4. `campaignConfig`라는 새 구성을 사용하여 캠페인을 만듭니다.
5. PutEvents에서 전송되는 이벤트를 수집할 이벤트 트래커를 만듭니다.
6. GetRecommendations를 호출하면 응답에 `RecommendationId`라는 새 필드가 반환됩니다.
7. `RecommendationId` 또는 사용자 지정 노출 항목 목록을 사용하여 putEvents를 호출합니다.
8. 캠페인이 업데이트될 때까지 기다립니다.
9. 자동 업데이트를 중지하도록 캠페인을 업데이트합니다.
10. 정리


> **참고:**: **이 노트북을 실행하는 데는 몇 시간 정도 걸립니다.**

### AWS API 액세스를 위한 액세스 키/비밀 키를 설정합니다.

사용하는 accessKey와 secretKey에 적절한 권한이 있는지 확인하세요. 또한 이 데모를 실행할 리전을 선택합니다.

In [ ]:
accessKeyId = ""
secretAccessKey = ""
region_name = ""

In [ ]:
import os
import boto3
from botocore.exceptions import ClientError
import time
import numpy as np
import pandas as pd
import json
from datetime import datetime

In [ ]:
suffix = str(np.random.uniform())[4:9]
prefix = 'user-personalization-'
print('prefix+suffix:{}{}'.format(prefix, suffix))
s3_bucket_name = (prefix + suffix).lower()
interaction_schema_name = prefix + 'interaction-'  + suffix
item_metadata_schema_name = prefix + 'items-'  + suffix
dataset_group_name = prefix + suffix
interaction_dataset_name = prefix + 'interactions-' + suffix
item_metadata_dataset_name = prefix + 'items-' + suffix
event_tracker_name = prefix + suffix
solution_name = prefix + suffix
event_tracker_name = prefix + suffix
campaign_name = prefix + suffix

### 1. 클라이언트 설정
먼저 Personalize와 S3를 위한 클라이언트를 설정하겠습니다.

In [ ]:
# Public s3 bucket owned by Personalize service which used to store the example dataset.

personalize_s3_bucket = "personalize-cli-json-models"
s3_client = boto3.Session(aws_access_key_id=accessKeyId,
                           aws_secret_access_key=secretAccessKey, region_name=region_name).client('s3')



####  Personalize 클라이언트 초기화

In [ ]:

personalize = boto3.Session(aws_access_key_id=accessKeyId,
                           aws_secret_access_key=secretAccessKey, region_name=region_name).client('personalize')
personalize_runtime = boto3.Session(aws_access_key_id=accessKeyId,
                           aws_secret_access_key=secretAccessKey, region_name=region_name).client('personalize-runtime')
personalize_events = boto3.Session(aws_access_key_id=accessKeyId,
                           aws_secret_access_key=secretAccessKey, region_name=region_name).client('personalize-events')

### 샘플 데이터 세트

이 데모에서는 편의상 Personalize에서 제공하는 샘플 데이터 세트를 사용합니다.
항목 메타데이터 하나와 상호 작용 데이터 세트 하나, 이렇게 2개의 데이터 세트가 제공됩니다. 먼저 데이터 세트를 로컬로 다운로드합니다.

In [ ]:
interaction_dataset_key = "sample-dataset/interactions-sample.csv"
items_dataset_key = "sample-dataset/items-with-creation-timestamp-sample.csv"
interactions_file = os.getcwd() + "/interaction_raw.csv"
items_metadata_file = os.getcwd() + "/items_raw.csv"
s3_client.download_file(personalize_s3_bucket, interaction_dataset_key, interactions_file)
s3_client.download_file(personalize_s3_bucket, items_dataset_key, items_metadata_file)

In [ ]:
interactions_df = pd.read_csv(interactions_file)
items_df = pd.read_csv(items_metadata_file)

#### 이제 상호 작용 데이터 프레임을 살펴보겠습니다.

In [ ]:
interactions_df.head(2)

#### 상호 작용 데이터 세트
**ITEM_ID**: EVENT_TYPE에 해당하는 항목입니다.

**EVENT_TYPE**: 이벤트 유형입니다.

**TIMESTAMP**: 상호 작용의 타임스탬프(밀리초)입니다. 단위가 밀리초라는 것을 유의하세요.

**USER_ID**: 이 노출에 해당하는 사용자 ID입니다.

**IMPRESSION:**  이제 상호 작용 데이터 세트에서 이벤트 데이터와 함께 노출 데이터를 선택적으로 전달할 수 있습니다. 위에서 보듯이 이 데이터는 `IMPRESSION`라는 새 필드를 통해 전달되며, 이 필드는 사용자가 상호 작용한 항목(예: 사용자에게 표시된 항목)의 파이프로 연결된 데이터를 받으며, 노출에는 클릭한 항목도 포함됩니다.


In [ ]:
items_df.head(2)

#### 항목 데이터 세트

항목 데이터 세트에는 Item_ids 및 관련 메타데이터가 포함되어 있습니다.

**ITEM_ID**: 항목 ID입니다.

**genres** 항목의 메타데이터입니다. 동일한 항목에 여러 범주 분류가 있는 경우 파이프(" | ")를 사용하여 연결합니다.

**creation_timestamp** 항목이 추가된 시간을 나타내는 타임스탬프입니다.

데이터 세트에 대한 자세한 내용은 설명서를 참조하세요.
https://docs.aws.amazon.com/personalize/latest/dg/how-it-works-dataset-schema.html

#### 타임스탬프 업데이트

여기서는 실시간 상호 작용이 추천에 미치는 영향을 보여주기 위해 데이터 세트의 타임스탬프를 오늘로부터 8일 전으로 업데이트하겠습니다.

In [ ]:
current_time = int(time.time())
one_hour_ago = current_time - 8 * 24 * 60 * 60
# Get the time gap between the latest timestamp in the interaction and the current time 
interactions_df = interactions_df.astype({"TIMESTAMP": 'int64'})
latest_time_in_csv = interactions_df["TIMESTAMP"].max()
delta = one_hour_ago - latest_time_in_csv

In [ ]:
# shift the latest timestamp in the interactions_df to be the last hour timestamp
interactions_df.TIMESTAMP = interactions_df.TIMESTAMP + delta
interactions_df.to_csv(os.getcwd() + "/interaction.csv", index = False)

# shift the latest timestamp in the items_df to be the last hour timestamp
items_df = items_df.astype({"creation_timestamp": 'int64'})
items_df.creation_timestamp = items_df.creation_timestamp + delta
items_df.to_csv(os.getcwd() + "/items.csv", index = False)

#### 업데이트 후 상호 작용 및 항목 데이터 세트를 한 번 더 확인합니다.

In [ ]:
interactions_df.head(2)

In [ ]:
items_df.head(2)


### 2. Amazon Personalize로 데이터 수집
이제 데이터 세트 그룹을 만들고, 스키마를 만들고, 데이터 세트를 업로드하고, 데이터 세트 가져오기 작업을 만듭니다. 이 작업은 이전과 동일합니다.

#### a. 데이터 세트 그룹 생성

이는 기존 레시피와 유사하며, 전체 설명서는 [여기](https://docs.aws.amazon.com/personalize/latest/dg/API_DatasetGroup.html)에서 확인할 수 있습니다.

In [ ]:
create_dataset_group_response = personalize.create_dataset_group(
    name = dataset_group_name
)
dataset_group_arn = create_dataset_group_response['datasetGroupArn']

In [ ]:

print('dataset_group_arn : {}'.format(dataset_group_arn))

In [ ]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(20)

#### b. 데이터 세트 스키마 생성

> **_참고:_**: `Impression` 필드에는 문자열 유형이 있으며 값이 여러 개인 경우 파이프 연결을 사용합니다.

In [ ]:
interaction_schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        { 
            "name": "EVENT_TYPE",
            "type": "string"
        },
        {
            "name": "IMPRESSION",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        },
        {
            "name": "USER_ID",
            "type": "string"
        },
    ],
    "version": "1.0"
}

In [ ]:
interaction_schema_response = personalize.create_schema(
    name = interaction_schema_name,
    schema = json.dumps(interaction_schema)
)
# print(json.dumps(create_schema_response, indent=2))
interaction_schema_arn = interaction_schema_response['schemaArn']
print('interaction_schema_arn:\n', interaction_schema_arn)

In [ ]:
item_metadata_schema = {
    "type": "record",
    "name": "Items",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
    {
        "name": "ITEM_ID",
        "type": "string"
    },
    {
        "name": "GENRES",
        "type": "string",
        "categorical": True
    },
    {
        "name": "CREATION_TIMESTAMP",
        "type": "long"
    }
    ],
    "version": "1.0"
}

item_metadata_schema_response = personalize.create_schema(
    name = item_metadata_schema_name,
    schema = json.dumps(item_metadata_schema)
)

# print(json.dumps(create_schema_response, indent=2))
item_metadata_schema_arn = item_metadata_schema_response['schemaArn']
print('item_metadata_schema_arn:\n', item_metadata_schema_arn)

#### c. 데이터 세트 생성

In [ ]:
interactions_dataset_response = personalize.create_dataset(
    datasetType = 'INTERACTIONS',
    datasetGroupArn = dataset_group_arn,
    schemaArn = interaction_schema_arn,
    name = interaction_dataset_name
)
interaction_dataset_arn = interactions_dataset_response['datasetArn']
#print(json.dumps(create_dataset_response, indent=2))
print('interaction_dataset_arn:\n', interaction_dataset_arn)

items_dataset_response = personalize.create_dataset(
    datasetType = 'ITEMS',
    datasetGroupArn = dataset_group_arn,
    schemaArn = item_metadata_schema_arn,
    name = item_metadata_dataset_name
)
item_metadata_dataset_arn = items_dataset_response['datasetArn']
#print(json.dumps(create_dataset_response, indent=2))
print('item_metadata_dataset_arn:\n', item_metadata_dataset_arn)

#### d. S3 버킷에 데이터 세트를 업로드하고 적절한 S3 버킷 정책, IAM 역할 등을 설정

이러한 데이터 세트를 업로드해야 합니다. 또는 데이터 세트가 이미 있는 버킷 이름을 제공할 수 있습니다.

In [ ]:
#!aws s3 mb s3://{s3_bucket_name}
s3_bucket_name

In [ ]:
s3_client.create_bucket(Bucket=s3_bucket_name,
                          CreateBucketConfiguration={
                              'LocationConstraint': region_name})

In [ ]:
interactions_file = os.getcwd() + "/interaction.csv"
items_metadata_file = os.getcwd() + "/items.csv"

In [ ]:
s3_client.upload_file(Filename=interactions_file, Bucket=s3_bucket_name,
    Key="interaction.csv")
s3_client.upload_file(Filename=items_metadata_file, Bucket=s3_bucket_name,
    Key="items.csv")

#### e. S3 버킷에 정책 연결

In [ ]:
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(s3_bucket_name),
                "arn:aws:s3:::{}/*".format(s3_bucket_name)
            ]
        }
    ]
}

s3_client.put_bucket_policy(Bucket=s3_bucket_name, Policy=json.dumps(policy));

#### f. Personalize가 데이터 세트에 액세스할 수 있도록 적절한 IAM 역할 설정

In [ ]:
iam = boto3.client(service_name='iam', 
                         aws_access_key_id = accessKeyId, 
                         aws_secret_access_key = secretAccessKey)   


role_name = "PersonalizeS3Role-"+suffix
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}
try:
    create_role_response = iam.create_role(
        RoleName = role_name,
        AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
    );

    iam.attach_role_policy(
        RoleName = role_name,
        PolicyArn = "arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
    );

    role_arn = create_role_response["Role"]["Arn"]
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        role_arn = iam.get_role(RoleName=role_name)['Role']['Arn']
    else:
        raise

In [ ]:
print('role_arn:', role_arn)

#### g. 데이터를 업로드하는 데이터 세트 가져오기 작업 생성

In [ ]:
time.sleep(20) # wait for RoleARN completion
interactions_dij_response = personalize.create_dataset_import_job(
    jobName =  prefix + 'interactions-dij-' + suffix,
    datasetArn = interaction_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(s3_bucket_name, 'interaction.csv')
    },
    roleArn = role_arn
)

interactions_dij_arn = interactions_dij_response['datasetImportJobArn']
print('interactions_dij_arn: ', interactions_dij_arn)
#print(json.dumps(interactions_dij_arn, indent=2))

items_dij_response = personalize.create_dataset_import_job(
    jobName =  prefix + 'items-dij-' + suffix,
    datasetArn = item_metadata_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(s3_bucket_name, 'items.csv')
    },
    roleArn = role_arn
)

items_dij_arn = items_dij_response['datasetImportJobArn']
print('items_dij_arn:', items_dij_arn)

In [ ]:
dataset_job_arns = [interactions_dij_arn, items_dij_arn]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time and len(dataset_job_arns) != 0:
    time.sleep(60)    
    for dij_arn in dataset_job_arns:
        describe_dataset_import_job_response = personalize.describe_dataset_import_job(
            datasetImportJobArn = dij_arn
        )
        dataset_import_job = describe_dataset_import_job_response["datasetImportJob"]
        status = None
        if "latestDatasetImportJobRun" not in dataset_import_job:
            status = dataset_import_job["status"]
            print("{} : {}".format(dij_arn, status))
        else:
            status = dataset_import_job["latestDatasetImportJobRun"]["status"]
            print("DIJ_ARN: {}, LatestDatasetImportJobRun: {}".format(dij_arn, status))
    
        if status == "ACTIVE" or status == "CREATE FAILED":
            dataset_job_arns.remove(dij_arn)


### 3. 솔루션 및 솔루션 버전 생성

'aws-user-personalization'을 사용하여 솔루션을 만듭니다. 이 레시피는 사용자에게 추천되는 새 항목과 이전 항목의 균형을 조정합니다.

In [ ]:
recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization"
max_time = time.time() + 3*60*60 # 3 hours
create_solution_response = None
while time.time() < max_time:

    try:
        create_solution_response = personalize.create_solution(name=solution_name, 
                                    recipeArn= recipe_arn, 
                                    datasetGroupArn = dataset_group_arn)

        solution_arn = create_solution_response['solutionArn']
        print('solution_arn: ', solution_arn)
        break;
    except personalize.exceptions.ClientError as e:
        if 'EVENT_INTERACTIONS' not in str(e):
            print(json.dumps(create_solution_response, indent=2))
            print(e)
            break

#### 솔루션 버전 생성

In [ ]:
create_solution_version_response = personalize.create_solution_version(solutionArn = solution_arn)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print('solution_version_arn:', solution_version_arn)

In [ ]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

### 4. 캠페인 생성
캠페인을 만들 때 itemExplorationConfig를 설정하여 콜드 항목 탐색 가중치와 탐색 경과 시간 컷오프를 구성할 수 있습니다. 우선은 explorationWeight를 0.9로 더 높게 설정하고 explorationItemAgeCutOff를 7로 설정할 수 있습니다. 즉, 항목 생성 시간이 7일 미만이면 모두 콜드 항목으로 간주되며, 새 항목에 대해 탐색을 더 많이 수행합니다.

#### 캠페인 생성

In [ ]:
create_campaign_response = personalize.create_campaign(
    name = prefix + suffix,
    solutionVersionArn = solution_version_arn,
    minProvisionedTPS = 1,
    campaignConfig = {
        "itemExplorationConfig": {
            "explorationWeight": "0.9",
            "explorationItemAgeCutOff": "7"
        }
    }
)

campaign_arn = create_campaign_response['campaignArn']
print('campaign_arn:', campaign_arn)

In [ ]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

In [ ]:
describe_campaign_response = personalize.describe_campaign(campaignArn = campaign_arn)
campaign_summary = describe_campaign_response["campaign"]
campaign_summary

### 5. GetRecommendations 호출
데모의 편의상, 입력 데이터 세트의 사용자 ID를 사용하여 getRecommendation 호출을 수행합니다.
> **_참고:_**: Personalize GetRecommendations에서 반환된 항목 목록에 해당하는 `RecommendationId`라는 새 필드가 응답에 포함되어 있습니다. 이 RecommendationId 를 전달하여 노출을 나타낼 수 있습니다.
또한 RecommendationId와 ImpressionList를 모두 전달하면 ImpressionList가 우선하며 시스템에서 사용됩니다.

In [ ]:
rec_response = personalize_runtime.get_recommendations(campaignArn = campaign_arn, userId = '101')
print(rec_response['recommendationId'])

In [ ]:
rec_response['itemList']

### 6. 이벤트 트래커 생성

PutEvents API를 사용하여 지정된 데이터 세트 그룹에 이벤트 데이터를 보낼 때 사용하는 이벤트 트래커를 만듭니다.

In [ ]:
even_tracker_response = personalize.create_event_tracker( 
    name=event_tracker_name,
    datasetGroupArn=dataset_group_arn
)
event_tracker_arn  = even_tracker_response['eventTrackerArn']
event_tracking_id = even_tracker_response['trackingId']
#print(json.dumps(even_tracker_response,indent=2))
print('eventTrackerArn:{},\n eventTrackingId:{}'.format(event_tracker_arn, event_tracking_id))

### 7. PutEvents를 통해 Personalize에 노출 데이터를 보냅니다.
Amazon Personalize는 두 가지 유형의 노출을 모델링할 수 있습니다.
1. 암시적 노출과 명시적 노출이 있습니다. 암시적 노출은 사용자의 세션 중에 발생하는 노출로, 사용자가 항목을 표시할 때마다 Amazon Personalize가 자동으로 기록합니다. RecommendationId(and 연산자에 의해 반환됨)를 미래의 PutEvents 요청에 대한 입력으로 포함하여 암시적 노출을 추천 워크플로에 반영할 수 있습니다.


2. 명시적 노출은 PutEvents 요청을 실행할 때 수동으로 입력하는 노출입니다. 예를 들어 GetRecommendations에서 반환되는 항목 중 일부를 사용할 수 없는 등의 이유로 표시하지 않는 경우 명시적 노출을 사용합니다.

> **참고:** 위와 같이 상호 작용 스키마에서 `impression`을 정의한 경우, GetRecommendations에서 반환된 항목 또는 사용자의 자체 항목에 대한 노출 목록을 보내야 합니다.
**recommendationId와 Impressions를 모두 포함할 경우 Amazon Personalize는 기본적으로 명시적 노출을 사용합니다.**



#### 그러면 이전에 추천된 항목을 노출로 포함해보겠습니다.

In [ ]:
personalize_events.put_events(
     trackingId = event_tracking_id,
     userId= '101',
     sessionId = '1',
     eventList = [{
     'sentAt': datetime.now().timestamp(),
     'eventType' : 'click',
     'itemId' : rec_response['itemList'][0]['itemId'],        
     'recommendationId': rec_response['recommendationId'],
     'impression': [item['itemId'] for item in rec_response['itemList']],
     }]
    )

#### 새 항목을 넣을 수도 있습니다.

'2xx'라는 새 항목 ID를 Personalize에 넣어 보겠습니다.

In [ ]:
personalize_events.put_events(
     trackingId = event_tracking_id,
     userId= '101',
     sessionId = '1',
     eventList = [{
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '240',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '241',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '242',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '243',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '244',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '245',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '246',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '247',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '248',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '249',
     }]
    )

### 8. updateMode를 사용하여 새 SolutionVersion 생성

put-events 실행 후, Personalize가 새 데이터를 수집하는 동안 15분 정도 기다린 후 업데이트 모드를 사용하여 새 솔루션 버전을 생성합니다.

In [ ]:
create_solution_version_response = personalize.create_solution_version(solutionArn = solution_arn, trainingMode = "UPDATE")

solution_version_after_update = create_solution_version_response['solutionVersionArn']
print('solution_version_after_update:', solution_version_arn)

In [ ]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_after_update
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

### 9. 캠페인 업데이트

업데이트를 통해 얻은 최신 솔루션 버전 ARN으로 캠페인을 업데이트합니다.

In [ ]:
campaign_arn_response = personalize.update_campaign(campaignArn=campaign_arn, solutionVersionArn=solution_version_after_update)
print('campaign_arn_response: ', campaign_arn_response)

In [ ]:
# Wait for campaign update to reflect the new solution-version
solutionVersionArn = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    solutionVersionArn = describe_campaign_response["campaign"]["solutionVersionArn"]
    print("Campaign solution version: {}".format(solutionVersionArn))
    
    if solutionVersionArn == solution_version_after_update:
        break
        
    time.sleep(60)

# wait 1 minutes
time.sleep(60)

In [ ]:
desc_campaign_response = personalize.describe_campaign(campaignArn = campaign_arn)['campaign']["solutionVersionArn"]
desc_campaign_response

### 솔루션 버전을 업데이트한 후 추천을 다시 실행합니다.

0.9의 높은 탐색 가중치를 설정했으므로 추천에 새로운 항목이 표시될 것으로 예상할 수 있습니다.

In [ ]:
rec_response = personalize_runtime.get_recommendations(campaignArn = campaign_arn, userId = '101')

In [ ]:
rec_response['itemList']

### 다른 explorationWeight를 사용하여 캠페인 업데이트

낮은 explorationWeight를 설정했으므로 추천 목록에 이전 항목이 더 많이 표시될 것으로 예상됩니다.체중.

In [ ]:
desc_campaign_response = personalize.describe_campaign(campaignArn = campaign_arn)['campaign']
desc_campaign_response

In [ ]:
campaign_arn_response = personalize.update_campaign(campaignArn=campaign_arn, campaignConfig = {
        "itemExplorationConfig": {
            "explorationWeight": "0.1",
            "explorationItemAgeCutOff": "7"
        }
    })


In [ ]:
# Wait for campaign update to reflect the new explorationWeight
explorationWeight = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    explorationWeight = describe_campaign_response["campaign"]["campaignConfig"]['itemExplorationConfig']['explorationWeight']
    print("Current Campaign  explorationWeight: {}".format(explorationWeight))
    
    if explorationWeight == "0.1":
        break
        
    time.sleep(60)

# wait 1 minutes
time.sleep(60)

### explorationWeight 업데이트 후

추천을 다시 실행하면 이전 항목이 더 많이 표시되는 것을 확인할 수 있습니다.

In [ ]:
rec_response = personalize_runtime.get_recommendations(campaignArn = campaign_arn, userId = '101')

In [ ]:
rec_response

#### 10. 리소스 삭제

리소스를 모두 생성한 후에는 모든 리소스를 정리합니다.

In [ ]:
personalize.delete_campaign(campaignArn=campaign_arn)
while len(personalize.list_campaigns(solutionArn=solution_arn)['campaigns']):
    time.sleep(5)

personalize.delete_solution(solutionArn=solution_arn)
while len(personalize.list_solutions(datasetGroupArn=dataset_group_arn)['solutions']):
    time.sleep(5)

for dataset in personalize.list_datasets(datasetGroupArn=dataset_group_arn)['datasets']:
    personalize.delete_dataset(datasetArn=dataset['datasetArn'])
while len(personalize.list_datasets(datasetGroupArn=dataset_group_arn)['datasets']):
    time.sleep(5)
    
personalize.delete_event_tracker(eventTrackerArn=event_tracker_arn)
personalize.delete_dataset_group(datasetGroupArn=dataset_group_arn)